In [1]:
import azureml.core
print("SDK version:", azureml.core.VERSION)

SDK version: 1.0.10


In [ ]:
from azureml.core.workspace import Workspace
ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 

      'Resource group: ' + ws.resource_group, sep = '\n')

In [3]:
from azureml.core import Datastore
# only need to do it once
ds2 = Datastore.register_azure_file_share(workspace=ws, 
                                         datastore_name='CTscands', 
                                         file_share_name='bloodcell',
                                         account_name='zbatchaistorage', 
                                         account_key='MmkzeKphqtgv83Z3DfieWNF40bdj53FQ7WebO2zPW73Jh3YOxOgGuAkj/gi0Bo/4K+awcWtlGvpDE9tIQvKYYw==',
                                         create_if_not_exists=False)

In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# choose a name for your cluster
cluster_name = "gpucluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6', 
                                                           max_nodes=4)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True)

# Use the 'status' property to get a detailed status for the current cluster. 
print(compute_target.status.serialize())

Found existing compute target
{'allocationState': 'Steady', 'allocationStateTransitionTime': '2019-02-28T13:46:20.126000+00:00', 'creationTime': '2019-01-08T10:23:55.033355+00:00', 'currentNodeCount': 0, 'errors': None, 'modifiedTime': '2019-01-08T10:25:35.793472+00:00', 'nodeStateCounts': {'idleNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0, 'preparingNodeCount': 0, 'runningNodeCount': 0, 'unusableNodeCount': 0}, 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'targetNodeCount': 0, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_NC6'}


In [5]:
import os

project_folder = './dist-keras-ctscan-exp'
os.makedirs(project_folder, exist_ok=True)

aml_config/                         keras_cnn_dicom.py
BatchAI_convert_trial_CTscan.ipynb  keras_cnn_pydicom.py
dist-keras-ctscan-exp/              keras-ctscan-exp/
dist_keras_ctscan.py                keras-ctscan-exp-feb28/
dist_keras_trial.ipynb              README.md
HousePrice/


In [20]:
import shutil

#shutil.copy('keras_cnn_dicom.py', project_folder)
shutil.copy('dist_keras_ctscan.py', project_folder)

'./dist-keras-ctscan-exp/dist_keras_ctscan.py'

In [6]:
from azureml.core import Experiment

experiment_name = 'dist-keras-tf-exp'
experiment = Experiment(ws, name=experiment_name)


In [7]:
cat ./dist-keras-ctscan-exp/dist_keras_ctscan.py

In [14]:
from azureml.train.dnn import TensorFlow
script_params={
    '--data': ds2.path(),
    '--epoch': 1,
    '--save_model':'/outputs'
}

estimator = TensorFlow(source_directory=project_folder,
                      compute_target=compute_target,
                      entry_script='dist_keras_ctscan.py',
                      script_params=script_params,
                      node_count=2,
                      process_count_per_node=1,
                      distributed_backend='mpi',    
                      pip_packages=['pydicom','keras','scikit-image','scikit-learn','scipy','argparse',
                                    'opencv-python','pillow','numpy', 'pandas','matplotlib'],
                      use_gpu=True)

In [15]:
run = experiment.submit(estimator)
print(run)

Run(Experiment: dist-keras-tf-exp,
Id: dist-keras-tf-exp_1551371714_bd0f8e56,
Type: azureml.scriptrun,
Status: Starting)


In [16]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

In [17]:
run.wait_for_completion(show_output=True)

RunId: dist-keras-tf-exp_1551371714_bd0f8e56

Streaming azureml-logs/20_image_build_log.txt

2019/02/28 16:35:23 Using acb_vol_bf4c6105-3416-46ae-8e09-bee79e18acf0 as the home volume
2019/02/28 16:35:23 Creating Docker network: acb_default_network, driver: 'bridge'
2019/02/28 16:35:24 Successfully set up Docker network: acb_default_network
2019/02/28 16:35:24 Setting up Docker configuration...
2019/02/28 16:35:25 Successfully set up Docker configuration
2019/02/28 16:35:25 Logging in to registry: zaml3818199666.azurecr.io
2019/02/28 16:35:27 Successfully logged into zaml3818199666.azurecr.io
2019/02/28 16:35:27 Executing step ID: acb_step_0. Working directory: '', Network: 'acb_default_network'
2019/02/28 16:35:27 Obtaining source code and scanning for dependencies...
2019/02/28 16:35:28 Successfully obtained source code and scanned for dependencies
2019/02/28 16:35:28 Launching container with name: acb_step_0
Sending build context to Docker daemon  40.96kB

Step 1/13 : FROM mcr.micros

  Complete output from command /azureml-envs/azureml_ab4fe2162f6cc59e0173793ac575f999/bin/python -u -c "import setuptools, tokenize;__file__='/tmp/pip-install-l10puasc/horovod/setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" bdist_wheel -d /tmp/pip-wheel-74mzq5xp --python-tag cp36:
  zip_safe flag not set; analyzing archive contents...
  pycparser.ply.__pycache__.lex.cpython-36: module references __file__
  pycparser.ply.__pycache__.lex.cpython-36: module MAY be using inspect.getsourcefile
  pycparser.ply.__pycache__.yacc.cpython-36: module references __file__
  pycparser.ply.__pycache__.yacc.cpython-36: module MAY be using inspect.getsourcefile
  pycparser.ply.__pycache__.yacc.cpython-36: module MAY be using inspect.stack
  pycparser.ply.__pycache__.ygen.cpython-36: module references __file__
  
  Installed /tmp/pip-install-l10puasc/horovod/.eggs/pycparser-2.19-py3.6.egg
  running bdist_wheel
  r

    Uninstalling setuptools-40.8.0:
      Successfully uninstalled setuptools-40.8.0
  Running setup.py install for horovod: started
    Running setup.py install for horovod: finished with status 'done'
#
# To activate this environment, use:
# > source activate /azureml-envs/azureml_ab4fe2162f6cc59e0173793ac575f999
#
# To deactivate an active environment, use:
# > source deactivate
#


Removing intermediate container 3a829ecd3ab7
 ---> ed8824f0d044
Step 9/13 : ENV PATH /azureml-envs/azureml_ab4fe2162f6cc59e0173793ac575f999/bin:$PATH
 ---> Running in 14d488831793
Removing intermediate container 14d488831793
 ---> a33e0815e963
Step 10/13 : ENV LD_LIBRARY_PATH /azureml-envs/azureml_ab4fe2162f6cc59e0173793ac575f999/lib:$LD_LIBRARY_PATH
 ---> Running in 3490b5e78db3
Removing intermediate container 3490b5e78db3
 ---> 7f188395ec41
Step 11/13 : COPY azureml-setup/spark_cache.py azureml-setup/log4j.properties /azureml-setup/
 ---> 855f8bd8e835
Step 12/13 : RUN if [ $SPARK_HOME ]; then /bin/bash

{'runId': 'dist-keras-tf-exp_1551371714_bd0f8e56',
 'target': 'gpucluster',
 'status': 'Finalizing',
 'startTimeUtc': '2019-02-28T16:50:40.603427Z',
 'properties': {'azureml.runsource': 'experiment',
  'ContentSnapshotId': 'cb95a289-a8d4-4edb-925d-a4b64cc6f38f'},
 'runDefinition': {'Script': 'dist_keras_ctscan.py',
  'Arguments': ['--data',
   '$AZUREML_DATAREFERENCE_ctscands',
   '--epoch',
   '1',
   '--save_model',
   '/outputs'],
  'SourceDirectoryDataStore': None,
  'Framework': 0,
  'Communicator': 5,
  'Target': 'gpucluster',
  'DataReferences': {'ctscands': {'DataStoreName': 'ctscands',
    'Mode': 'Mount',
    'PathOnDataStore': None,
    'PathOnCompute': None,
    'Overwrite': False}},
  'JobName': None,
  'AutoPrepareEnvironment': True,
  'MaxRunDurationSeconds': None,
  'NodeCount': 2,
  'Environment': {'Python': {'InterpreterPath': 'python',
    'UserManagedDependencies': False,
    'CondaDependencies': {'name': 'project_environment',
     'dependencies': ['python=3.6.2',
